In [15]:
# Import the modules we will be using
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#import mplsoccer to demo creating a pitch on gridsearch
from mplsoccer import Pitch
from mplsoccer import VerticalPitch
import sqlite3
from highlight_text import fig_text, ax_text

import requests
import bs4
import json
from PIL import Image
import urllib
from analysis_tools import fotmob_visuals as fotmobvis
from analysis_tools import  whoscored_visuals as whovis

In [16]:
from analysis_tools import whoscored_data_engineering as who_eng

Data proccessing

In [17]:
# Set the font as the default font for all plots
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
font_dir = './fonts'
font_path = f"{font_dir}/Raleway/static/Raleway-ExtraLightItalic.ttf"
font = FontProperties(fname=font_path, size=12)
plt.rcParams['font.family'] = font.get_name()




In [18]:
data = pd.read_csv('./Data/liverpool03_05_23.csv')


In [19]:
from ast import literal_eval

data['qualifiers'] = [literal_eval(x) for x in data['qualifiers']]
data['satisfiedEventsTypes'] = [literal_eval(x) for x in data['satisfiedEventsTypes']]



In [20]:
for index, row in enumerate(data['qualifiers']):
    for element in row:
        if isinstance(element, dict) and ('ThrowIn' in element.keys() or 'CornerAwarded' in element.keys()):
            data.at[index, 'is_throw_or_corner'] = True
        else:
            data.at[index, 'is_throw_or_corner'] = False


In [21]:
data.is_throw_or_corner

0         NaN
1         NaN
2       False
3       False
4       False
        ...  
1587      NaN
1588      NaN
1589      NaN
1590    False
1591    False
Name: is_throw_or_corner, Length: 1592, dtype: object

In [22]:
def get_passes_df(events_dict):
    df = pd.DataFrame(events_dict)
    # create receiver column based on the next event
    # this will be correct only for successfull passes
    df["pass_recipient"] = df["playerName"].shift(-1)
    # filter only passes

    passes_ids = df.index[df['event_type'] == 'Pass']
    df_passes = df.loc[
        passes_ids, ["id","minute", "x", "y", "endX", "endY", "teamId", "playerId","playerName", "event_type", "outcomeType","pass_recipient"]]

    return df_passes

In [23]:
passes_df = data[data['teamId']==26]

In [24]:
passes_df = get_passes_df(passes_df)
passes_df

,id,minute,x,y,endX,endY,teamId,playerId,playerName,event_type,outcomeType,pass_recipient
4,2.516083e+09,0.0,19.9,30.3,28.5,45.1,26,345957.0,Ibrahima Konaté,Pass,Successful,Jordan Henderson
5,2.516083e+09,0.0,28.5,45.1,32.3,40.0,26,68659.0,Jordan Henderson,Pass,Unsuccessful,Harvey Elliott
11,2.516083e+09,0.0,35.7,33.0,39.4,35.4,26,345957.0,Ibrahima Konaté,Pass,Successful,Fabinho
12,2.516083e+09,0.0,39.4,35.4,30.2,6.5,26,115916.0,Fabinho,Pass,Successful,Trent Alexander-Arnold
13,2.516083e+09,0.0,30.3,5.2,66.4,10.7,26,318871.0,Trent Alexander-Arnold,Pass,Unsuccessful,Cody Gakpo
...,...,...,...,...,...,...,...,...,...,...,...,...
1570,2.516159e+09,92.0,55.4,57.6,52.7,80.8,26,345957.0,Ibrahima Konaté,Pass,Successful,Virgil van Dijk
1571,2.516159e+09,92.0,52.7,80.8,27.7,51.5,26,95408.0,Virgil van Dijk,Pass,Successful,Alisson
1573,2.516159e+09,92.0,29.7,54.9,63.5,97.4,26,114147.0,Alisson,Pass,Successful,Andrew Robertson
1575,2.516159e+09,92.0,68.8,96.6,92.6,59.3,26,115726.0,Andrew Robertson,Pass,Successful,Diogo Jota


In [25]:
offensive_actions = who_eng.find_offensive_actions(data)
offensive_actions =  offensive_actions[(offensive_actions['event_type'] != 'Pass') & (offensive_actions['event_type'] != 'BallTouch')]

offensive_actions

,Unnamed: 0,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,index,id,match_string,team_name,Venue,xThreat,xThreat_gen,playerName,is_throw_or_corner,is_carry_into_box
35,48,32.5,2.0,33.5,26,70.9,16.7,2.0,1.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.002426,0.002426,Harvey Elliott,False,NaN
37,50,33.5,2.0,36.5,26,85.2,3.5,2.0,1.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.008566,0.008566,Mohamed Salah,False,NaN
39,52,35.0,2.0,40.0,26,86.6,32.4,2.0,1.0,NaN,...,50.0,2.516085e+09,Liverpool - Man Utd,Liverpool,Home,NaN,NaN,Harvey Elliott,False,NaN
47,62,40.5,3.0,24.5,26,24.2,17.0,3.0,1.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.000947,0.000947,Fabinho,False,NaN
62,84,51.5,4.0,23.0,26,79.5,68.4,4.0,1.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.074598,0.074598,Mohamed Salah,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,1555,858.5,91.0,41.0,26,80.8,83.8,93.0,2.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.009758,0.009758,James Milner,False,NaN
1143,1558,860.5,91.0,49.0,26,70.8,72.6,93.0,2.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,-0.001950,0.000000,Stefan Bajcetic,False,NaN
1147,1562,863.5,91.0,55.0,26,74.1,89.4,93.0,2.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.002149,0.002149,James Milner,False,NaN
1155,1572,868.5,92.0,12.5,26,27.7,51.5,94.0,2.0,"{'value': 99, 'displayName': 'Carry'}",...,NaN,NaN,NaN,NaN,NaN,0.000302,0.000302,Alisson,False,NaN


In [26]:
defensive_actions = who_eng.find_defensive_actions(data)

Defenders Dashboard

In [27]:


fig = plt.figure(figsize=(14, 10), constrained_layout=True, dpi=600)
gs = fig.add_gridspec(ncols=2, nrows=2)
fig.set_facecolor("#201D1D")
# create subplots using gridspec
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 0])

axes = [ax1, ax2, ax3, ax4]

# apply modifications to all subplots
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_facecolor("#201D1D")#%%


whovis.plot_passmap_opta(ax1,'Virgil van Dijk',data)

whovis.plot_convex_hull_opta_side(ax2,'Virgil van Dijk',data)

whovis.plot_players_defensive_actions_opta(ax3,'Virgil van Dijk',defensive_actions)

whovis.plot_carry_opta(ax4, 'Virgil van Dijk', data)


AttributeError: module 'analysis_tools.whoscored_visuals' has no attribute 'plot_passmap_opta'

Midfielders Dashboard

In [ ]:
fig = plt.figure(figsize=(12, 10), constrained_layout=True, dpi=600)
gs = fig.add_gridspec(ncols=2, nrows=2)
fig.set_facecolor("#201D1D")
# create subplots using gridspec
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 0])

axes = [ax1, ax2, ax3, ax4]

# apply modifications to all subplots
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_facecolor("#201D1D")


whovis.plot_player_passmap_opta(ax1,'Harvey Elliott',data)


whovis.plot_convex_hull_opta_player(ax2,'Harvey Elliott',data)

whovis.plot_players_offensive_actions_opta(ax3,'Harvey Elliott',offensive_actions)

whovis.plot_carry_into_box_player_opta(ax4, 'Harvey Elliott', data)


Attackers Dashboard

In [ ]:
fig = plt.figure(figsize=(12, 10), constrained_layout=True, dpi=600)
gs = fig.add_gridspec(ncols=3, nrows=2)
fig.set_facecolor("#201D1D")

# create subplots using gridspec
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, 0])


axes = [ax1, ax2, ax3, ax4]

# apply modifications to all subplots
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_facecolor("#201D1D")
    ax.axis('off')



whovis.plot_player_shotmap(ax1,3901187,'Mohamed Salah')

whovis.plot_convex_hull_opta_side(ax2,'Mohamed Salah',data)

whovis.plot_players_offensive_actions_opta(ax3,'Mohamed Salah',offensive_actions)

whovis.plot_carry_opta(ax4, 'Mohamed Salah', data)

#whovis.plot_players_defensive_actions_opta(ax1,'Mohamed Salah',defensive_actions)

player_id = 292462
fotmob_url = 'https://images.fotmob.com/image_resources/playerimages/{player_id}.png'
club_icon = Image.open(urllib.request.urlopen(f'{fotmob_url}'))

logo_ax = fig.add_axes([0, 1, 0.2, 0.2], frameon=False)
logo_ax.imshow(club_icon, aspect='equal')
ax.axis('off')


In [28]:
fig = plt.figure(figsize=(15, 13), constrained_layout=True, dpi=800)
gs = fig.add_gridspec(ncols=3, nrows=3)  # change nrows from 2 to 3
fig.set_facecolor("#201D1D")

# create subplots using gridspec
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])  # add new subplot
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])  # add new subplot
ax6 = fig.add_subplot(gs[1, 2])  # add new subplot


axes = [ax1, ax2, ax3, ax4,ax5,ax6]

# apply modifications to all subplots
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_facecolor("#201D1D")
    ax.axis('off')


ax1.set_title('Shot Map', color ='#e1dbd6'  , fontsize=16, pad=10)
ax4.set_title('Ball Carries',color ='#e1dbd6'  , fontsize=16, pad=10)
ax3.set_title('Offensive Actions',color ='#e1dbd6'  , fontsize=16, pad=10)
ax2.set_title('Territory map', color ='#e1dbd6'  ,fontsize=16, pad=10)
ax5.set_title('Pass Map', color ='#e1dbd6'  ,fontsize=16, pad=10)
ax6.set_title('Passes Receive ', color ='#e1dbd6'  ,fontsize=16, pad=10)



fotmobvis.plot_player_shotmap(ax1,3901187,'Mohamed Salah')

whovis.plot_carry_into_box_player_opta(ax4,'Mohamed Salah',data)

whovis.plot_players_offensive_actions_opta(ax3,'Mohamed Salah',offensive_actions)

whovis.plot_convex_hull_opta_player(ax2, 'Mohamed Salah', data)

whovis.plot_player_passmap_opta(ax5,'Mohamed Salah',passes_df)
#whovis.plot_players_defensive_actions_opta(ax5,'Mohamed Salah',defensive_actions)

whovis.plot_player_passes_rec_opta(ax6,'Mohamed Salah',passes_df)


player_id = 292462
player_logo_path = f'Data/player_image/{player_id}.png'
club_icon = Image.open(player_logo_path).convert('RGBA')

logo_ax = fig.add_axes([0, .94, 0.12, 0.12], frameon=False)
logo_ax.imshow(club_icon)
logo_ax.set_xticks([])
logo_ax.set_yticks([])

fig.suptitle('Salah Post Match Dashboard', fontsize=24, color ='#e1dbd6' , ha='center')



Text(0.5, 0.98, 'Salah Post Match Dashboard')

findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: Font family 'Raleway' not found.
findfont: F

In [ ]:



ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

ax6.grid(False)
ax6.spines['top'].set_visible(False)
ax6.spines['right'].set_visible(False)
ax6.spines['bottom'].set_visible(False)
ax6.spines['left'].set_visible(False)
whovis.plot_offensive_actions(ax1, 1640885, 26, data)


whovis.plot_offensive_actions(ax2, 1640885, 32, data)

whovis.plot_defensive_actions_opp_half(ax4, 1640885, 32, data)


whovis.plot_defensive_actions_opp_half(ax3, 1640885, 26, data)



team_id = liv['teamId'].iloc[0]
team_logo_path = f'Data/team_logo/{team_id}.png'
club_icon = Image.open(team_logo_path).convert('RGBA')

away_team_id = away['teamId'].iloc[0]
away_team_logo_path = f'Data/team_logo/{away_team_id}.png'
away_club_icon = Image.open(away_team_logo_path).convert('RGBA')


#away_logo_ax = ax.inset_axes([0.75, 0.8, 0.2, 0.2], transform=ax.transAxes)
away_logo_ax = fig.add_axes([.90, 1, .2, .2], frameon=False)


away_logo_ax.imshow(away_club_icon, aspect='equal')
away_logo_ax.axis('off')

away_logo_ax.set_xticks([])
away_logo_ax.set_yticks([])


logo_ax = fig.add_axes([0, 1, 0.2, 0.2], frameon=False)
logo_ax.imshow(club_icon, aspect='equal')
logo_ax.set_xticks([])
logo_ax.set_yticks([])


#logo_ax = fig.add_axes([0, 1, 0.1, 0.1], frameon=False)
#logo_ax.imshow(club_icon)
logo_ax.axis('off')



In [ ]:



ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

ax6.grid(False)
ax6.spines['top'].set_visible(False)
ax6.spines['right'].set_visible(False)
ax6.spines['bottom'].set_visible(False)
ax6.spines['left'].set_visible(False)
whovis.plot_offensive_actions(ax1, 1640885, 26, data)


whovis.plot_offensive_actions(ax2, 1640885, 32, data)

whovis.plot_defensive_actions_opp_half(ax4, 1640885, 32, data)


whovis.plot_defensive_actions_opp_half(ax3, 1640885, 26, data)


fotmob_url = 'https://images.fotmob.com/image_resources/playerimages/'
club_icon = Image.open(urllib.request.urlopen(f'{fotmob_url}{player_id:.0f}.png'))
if grayscale:
    club_icon = club_icon.convert('LA')
ax.imshow(club_icon)
ax.axis('off')
return ax




team_id = liv['teamId'].iloc[0]


away_team_id = away['teamId'].iloc[0]
away_team_logo_path = f'Data/team_logo/{away_team_id}.png'
away_club_icon = Image.open(away_team_logo_path).convert('RGBA')


#away_logo_ax = ax.inset_axes([0.75, 0.8, 0.2, 0.2], transform=ax.transAxes)
away_logo_ax = fig.add_axes([.90, 1, .2, .2], frameon=False)


away_logo_ax.imshow(away_club_icon, aspect='equal')
away_logo_ax.axis('off')

away_logo_ax.set_xticks([])
away_logo_ax.set_yticks([])




#logo_ax = fig.add_axes([0, 1, 0.1, 0.1], frameon=False)
#logo_ax.imshow(club_icon)
logo_ax.axis('off')

